# 1 Overview of Speech Recognition
## 1.1 Hidden Markov Model for Accoustic Modeling
As speech is 


# 2 Hidden Markov Model for Accoustic Modeling
## Phoneme model of words


# 2.1 Gaussian Mixture Model
## Overview


+ **Mixture of Gaussians**
$$p(x) = \sum_{k=1}^{K} \pi_{k}N(x|\mu_{k}, \Sigma_{k})$$
> + Add a hidden variable Z, which is like a swith to each Gaussian
  $$p(Z) = \prod_{k=1}^{K} \pi_{k}^{z_{k}}$$
  + each Gaussian can be expressed as x depends on $z_{k}$
  $$p(x | z_{k} = 1) = N(x|\mu_{k}, \Sigma_{k})$$
  and
  $$p(x | Z) = \prod_{k=1}^{K} N(x|\mu_{k}, \Sigma_{k})^{z_{k}}$$
  + Marginal Distribution of x
  $$p(x) = \sum_{z} p(x|z) p(z) = \sum_{k=1}^{K} \pi_{k} N(x|\mu_{k}, \Sigma_{k})$$
  + Responsity of a Gaussion
  $$\begin{aligned}\gamma(z_{k}) \equiv p(z_{k}=1|x) = \frac{p(z_{k}=1)p(x|z_{k}=1)}{\sum_{j=1}^{K} p(z_{j}=1)p(x|z_{j}=1)} 
     = \frac{\pi_{k}N(x|\mu_{k}, \Sigma_{k})}{\sum_{j=1}^{K} \pi_{j}N(x|\mu_{j}, \Sigma_{j})} \end{aligned}$$
  

+ **EM**
> + likelihood function
    $$ p(X|\pi, \mu, \Sigma) = \prod_{n=1}^{N} p(x_{n}|\pi,\mu,\Sigma) 
    = \prod_{n=1}^{N}\sum_{k=1}^{K}\pi_{k}N(x_{n}|\mu_{k}, \Sigma_{k}) $$
    Take log and get log-likelihood
    $$\ln p(X|\pi, \mu, \Sigma) = \sum_{n=1}^{N} \ln \bigg\{ \sum_{k=1}^{K}\pi_{k}N(x_{n}|\mu_{k}, \Sigma_{k}) \bigg\} $$
  + set Derivative of the log-likelihood w.r.t $\pi$, $\mu$, $\Sigma$ to ***Zero*** <br/>
    Since there is a constraint with $\pi$: $\sum_{\pi_{k}}^{K} = 1$, we should use ***Langrange multiplier*** method.
    >> + Derivative of log-likelihood w.r.t $\mu_{k}$
    $$N(x|\mu, \Sigma) = \frac{1}{(2\pi)^\frac{D}{2}|\Sigma|^{\frac{1}{2}}} 
    exp\bigg\{ -\frac{1}{2}(x-\mu)^{T}\Sigma^{-1}(x-\mu)  \bigg\}$$
    $$\nabla_{\mu_{k}} \ln p(X|\pi, \mu, \Sigma) = 
    \sum_{n=1}^{N} \frac{\pi_{k}N(x_{n}|\mu_{k}, \Sigma_{k})}{\Sigma_{j=1}^{K} \pi_{j}N(x_{n}|\mu_{j}, \Sigma_{n})}
    \Sigma_{k}^{-1} (x_{n} - \mu_{k}) = 0 $$    
    $$\mu_{k} = \frac{1}{N_{k}} \sum_{n=1}^{N} \gamma (z_{nk}) x_{n}$$
    where $N_{k} = \sum_{n=1}{N} \gamma (z_{nk}) $
    + Derivative w.r.t $\Sigma_{k}$   
    $$\Sigma_{k} = \frac{1}{N_{k}} \sum_{n=1}^{N} \gamma(z_{nk}) (x_n-\mu_k)(x_n-\mu_k)^T$$

    
    
         
    
  

+ **VEM**
> + Variational Method

+ **Choose of Covariance** 
> + Diagonal
> + Full



In [139]:
import numpy as np
from numpy import linalg as la
def Gauss(x, mu, Sigma):
    """Multivariate diagnal covariance Gaussian"""
    return np.exp(-np.dot(np.dot(x-mu, la.inv(Sigma)), x-mu)/2) / (np.power(2 * np.pi, len(mu)/ 2) * np.sqrt(la.det(Sigma)))

def square_dot(c):
    """ a * a' """
    return np.dot(c[:,np.newaxis], c[:,np.newaxis].T)
class GMM:
    def __init__(self,Pis_init, Mus_init, Sigmas_init, iters):
        assert len(Pis_init) == len(Mus_init) == len(Sigmas_init)
        self.K = len(Pis_init)
        self.Pis = Pis_init
        self.Mus = Mus_init
        self.Sigmas = Sigmas_init
        self.iters = iters
        
    def Gausses(self, x):
        #return np.array([self.Pis[i] * Gauss(x, self.Mus[i], self.Sigmas[i]) for i in range(self.K)])
        return np.array([pi * Gauss(x, mu, Sigma) for (pi, mu, Sigma) in zip(self.Pis, self.Mus, self.Sigmas)])
    
    def E(self, x):      
        g = self.Gausses(x)
        return g/np.sum(g)
        
    def EM(self, X):
        N = len(X)
        # E step
        R_nk = np.array([self.E(x) for x in X])
        
        # M step
        N_k  = np.sum(R_nk, axis=0)
        self.Pis = N_k/N
        # old_Mus = self.Mus
        self.Mus = [np.sum([R_nk[n, k] * X[n] for n in range(N)], axis=0)/N_k[k] for k in range(self.K)]
        Mus = self.Mus
        self.Sigmas = [np.sum([R_nk[n,k] * square_dot(X[n]-Mus[k]) for n in range(N)], axis=0) /N_k[k] for k in range(self.K)]
    
    def params(self):
        return [self.Pis, self.Mus, self.Sigmas]
    def train(self, X):
        for i in range(self.iters):
            self.EM(X)
            

In [145]:
Pis = [.3, 0.7]
Mus = [np.array([0, 0]), np.array([1,0])]
Sigmas = [np.array([[1,0], [0,1]]), np.array([[1,0], [0,1]])]
x = np.array([[2,0], [0,0.5], [1,1]])

gmm = GMM(Pis, Mus, Sigmas, 4)

gmm.train(x)



In [73]:
x

array([[ 2. ,  0. ],
       [ 0. ,  0.5],
       [ 1. ,  1. ]])

In [146]:
gmm.params()

[array([ 0.66609022,  0.33390978]),
 [array([ 0.4995783 ,  0.74978915]),
  array([  1.99825168e+00,   1.71537899e-03])],
 [array([[ 0.24999982,  0.12499991],
         [ 0.12499991,  0.06249996]]), array([[ 0.00178919, -0.00172336],
         [-0.00172336,  0.00170695]])]]

In [119]:
a = np.dot((x[0]-Mus[0])[:, np.newaxis], (x[0]-Mus[0])[:, np.newaxis].T)

In [123]:
np.sum([a,a], axis=0)

array([[ 4.,  0.],
       [ 4.,  0.]])

## 2.2 GMM in sklearn

In [1]:
import numpy as np
from scipy import linalg
from sklearn import mixture
n_comp = 64
gmm = mixture.GMM(n_components=n_comp, covariance_type='diag')




In [2]:
(np.array([1,2,3]) / (np.array([1,2,3]))).dot(np.array([1,2,3]))
np.prod(np.array([1,2,3]))
np.dot(np.array([1,2,3]).T, np.array([1,2,3]))

14

# 3 Hidden Markov Model
## 3.1 Markov Chains
+ Markov Chains property
> The current value is only dependent on the recent one
$$p(x_n | x_1,x_2,...,x_{n-1}) = p(x_n|x_{n-1})$$
$$p(x_1,x_2, ...,x_N) = p(x_1)\prod_{n=2}^{N}p(x_n|x_{n-1})$$

## 3.2 Hidden States


$$ $$


